<a href="https://colab.research.google.com/github/ngdeva99/MottaBOSS/blob/master/Models/custom_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
!unzip '/gdrive/My Drive/0_split.zip'

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam,Adadelta
import numpy as np

In [ ]:
datagen = ImageDataGenerator()

In [ ]:
train_it = datagen.flow_from_directory('0_split/train/', class_mode='categorical', batch_size=128)
# load and iterate validation dataset
val_it = datagen.flow_from_directory('0_split/validation/', class_mode='categorical', batch_size=128)
# load and iterate test dataset
test_it = datagen.flow_from_directory('0_split/test/', class_mode='categorical', batch_size=128)


Found 53484 images belonging to 4 classes.
Found 7182 images belonging to 4 classes.
Found 14364 images belonging to 4 classes.


In [ ]:
AlexNet = Sequential()

#1st Convolutional Layer
AlexNet.add(Conv2D(filters=16, input_shape=(256,256,3), kernel_size=(3,3), strides=(2,2), padding='valid'))
AlexNet.add(Activation('relu'))

AlexNet.add(Conv2D(filters=16, kernel_size=(3,3), strides=(2,2), padding='valid'))
AlexNet.add(Activation('relu'))

#2nd Convolutional Layer
AlexNet.add(Conv2D(filters=32, kernel_size=(3,3), strides=(2,2), padding='valid'))
AlexNet.add(Activation('relu'))

#3rd Convolutional Layer
AlexNet.add(Conv2D(filters=32, kernel_size=(3,3), strides=(2,2), padding='valid'))
AlexNet.add(Activation('relu'))

# 1st Max pooling Layer
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(1,1), padding='valid'))
AlexNet.add(Dropout(0.25))

#3rd Convolutional Layer
AlexNet.add(Conv2D(filters=64, kernel_size=(3,3), strides=(2,2), padding='valid'))
AlexNet.add(Activation('relu'))

#4th Convolutional Layer
AlexNet.add(Conv2D(filters=64, kernel_size=(3,3), strides=(2,2), padding='valid'))
AlexNet.add(Activation('relu'))

# 2nd Max pooling Layer
AlexNet.add(MaxPooling2D(pool_size=(2,2), strides=(1,1), padding='valid'))
AlexNet.add(Dropout(0.25))

AlexNet.add(Flatten())

######################
# 1st Fully Connected Layer
#AlexNet.add(Dense(512, input_shape=(224*224*3,)))
#######################

AlexNet.add(Dense(512))
AlexNet.add(Activation('relu'))
AlexNet.add(Dropout(0.50))

# Output Layer
AlexNet.add(Dense(4))
AlexNet.add(Activation('softmax'))

#Model Summary
AlexNet.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 127, 127, 16)      448       
_________________________________________________________________
activation_1 (Activation)    (None, 127, 127, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 63, 63, 16)        2320      
_________________________________________________________________
activation_2 (Activation)    (None, 63, 63, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 31, 31, 32)        4640      
_________________________________________________________________
activation_3 (Activation)    (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 15, 15, 32)       

In [ ]:
opt = Adam(lr = 0.001)

In [ ]:
AlexNet.compile(loss = keras.losses.categorical_crossentropy, optimizer=opt, metrics=['accuracy'])

In [ ]:
AlexNet.fit_generator(train_it, steps_per_epoch=16, validation_data=val_it, validation_steps=16, epochs=50)

In [ ]:
AlexNet.evaluate_generator(test_it, steps=24)

# VGG 19 - 1024 extra 

In [19]:
from keras.applications import VGG19

In [20]:
base_model = VGG19(include_top = False, weights = 'imagenet', input_shape = (256,256,3), classes = 4)

80142336/80134624 [==============================] - 2s 0us/step


In [ ]:
model= Sequential()
model.add(base_model) 
model.add(Flatten())
model.add(Dense(1024,activation=('relu')))
model.add(Dropout(.3))
#model.add(Dropout(.2))
model.add(Dense(4,activation=('softmax'))) 

model.summary()

In [39]:
## HYPER PARAMTERES
from keras.optimizers import SGD
batch_size= 100
epochs=50
learn_rate=0.001
sgd=SGD(lr=learn_rate,momentum=.9,nesterov=False)
adam=Adam(lr=learn_rate)

In [40]:
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

In [28]:
from keras.callbacks import ReduceLROnPlateau
lrr= ReduceLROnPlateau(monitor='val_acc', factor=.01, patience=3, min_lr=1e-5)

In [ ]:

model.fit_generator(train_it, steps_per_epoch=16, validation_data=val_it, validation_steps=16, epochs=32)

In [ ]:
model.evaluate_generator(test_it,steps=24
               )

STANDARD VGG19 - Transfer Learning

In [44]:
vgg_model = VGG19(include_top = False, weights = 'imagenet', input_shape = (256,256,3), classes = 4)

In [ ]:
vgg_std_model= Sequential()
vgg_std_model.add(base_model) 
vgg_std_model.add(Flatten())
vgg_std_model.add(Dropout(.3))
vgg_std_model.add(Dense(4,activation=('softmax'))) 

vgg_std_model.summary()

In [47]:
model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
history_vgg_std = model.fit_generator(train_it, steps_per_epoch=16, validation_data=val_it, validation_steps=16, epochs=50)